In [1]:
import gensim
import os
import collections
import smart_open
import random
import pickle

In [2]:
import re
import nltk
import sys
from nltk.stem import *
from nltk.corpus import stopwords

In [3]:
def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [4]:
def process(parts):
    for part in parts:
            pattern = r'^<.*> <.*> "'
            for p in re.findall(pattern,part):
                part = part.replace(p,' ').lower()
            pattern = r'"@.*$'
            for p in re.findall(pattern,part):
                part = part.replace(p,' ')
            pattern = r'\\u[0-9]*'
            for p in re.findall(pattern,part):
                part = part.replace(p,' ')
            pattern = r'([^a-zA-Z_ ])'#'|()'
            for p in re.findall(pattern,part):
                part = part.replace(p,' ')
            pattern = r' [a-zA-Z]{1} '
            for p in re.findall(pattern,part):
                part = part.replace(p,' ')
            pattern = r' [a-zA-Z]{1}$'
            for p in re.findall(pattern,part):
                part = part.replace(p,' ')
            pattern = r'^[a-zA-Z]{1} '
            for p in re.findall(pattern,part):
                part = part.replace(p,' ')

    return part

In [5]:
def custom_preprocess(line):
    stopwords_list=stopwords.words("english")
    stemmer = PorterStemmer()
    parts = [p for p in line.split("\t")]
    article_text = process(parts[1:])
    article_text = article_text.rstrip()
    article=[]
    for label in parts[0].split(","):
        for word in article_text.split(" "):
            if(word.rstrip()!="" and not word in stopwords_list):
                article.append(stemmer.stem(word))
    return " ".join(article)

In [6]:
def read_corpus(fname, tokens_only=False):
    with smart_open.smart_open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            if tokens_only:
                refined_line = custom_preprocess(line)
                yield gensim.utils.simple_preprocess(refined_line)
            else:
                # For training data, add tags
                refined_line = custom_preprocess(line)
                yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(refined_line), [i])

In [7]:
train_filename=r"/home/vikrambhatt/Academics/Machine Learning with Large datasets/Assignment2/DBPedia.verysmall/verysmall_train.txt"
test_filename=r"/home/vikrambhatt/Academics/Machine Learning with Large datasets/Assignment2/DBPedia.verysmall/verysmall_test.txt"
dev_filename=r"/home/vikrambhatt/Academics/Machine Learning with Large datasets/Assignment2/DBPedia.verysmall/verysmall_devel.txt"

In [8]:
train_corpus = list(read_corpus(train_filename))
test_corpus=list(read_corpus(test_filename,True))
dev_corpus =list(read_corpus(dev_filename, True))

In [9]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=40)

In [10]:
model.build_vocab(train_corpus)

In [11]:
%time model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

CPU times: user 1min 16s, sys: 1.62 s, total: 1min 18s
Wall time: 29.7 s


In [12]:
model.save("Train_Corupus.model")
save_obj(test_corpus,"TestCorpus")
save_obj(dev_corpus,"DevCorpus")

In [14]:
save_obj(train_corpus,"TrainCorpus")